Install Py Torch library

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Install Detectron 2 

The code will ask to restart the runtime to restart the runtime from Runtime > Restart runtime 


In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

Link your Google Drive you can skip this if your not using google drive for Model file (.pth) and inference input video

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importing the Detectron and Pytorch libraries

In [ ]:

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Importing the Predictor aong with the Configuration.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("train","train1")
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.CLIP_GRADIENTS.ENABLED=True
cfg.INPUT.CROP.ENABLED = True
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 25000  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (Person, Helmet, Car)
cfg.MODEL.DEVICE='cuda'
cfg.OUTPUT_DIR = "/content/outputs"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
# trainer.resume_or_load(resume=True)
# trainer.train()

Loading weigths from the Given path

In [ ]:
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/model_final.pth"#os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("train", )
predictor = DefaultPredictor(cfg)

**Inference**
<br>
Need to give the Pah to the input video and the file extesntion recomended to use **.mp4** **

In [ ]:
predictor = DefaultPredictor(cfg)
from detectron2.utils.visualizer import ColorMode

filenames =['/content/cardamage.mp4']
filename = filenames[0]
cap = cv2.VideoCapture(filename)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = cap.get(cv2.CAP_PROP_FPS)
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
basename = os.path.basename(filename)
# frame_width = int(cap.get(3)) 
# frame_height = int(cap.get(4)) 
   
size = (width, height) 
result = cv2.VideoWriter('output_video.mp4',  
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         frames_per_second, size)

print(num_frames)
print(size)
for _ in range(num_frames):
	ret, im = cap.read()
	outputs = predictor(im)
	v = Visualizer(im[:, :, ::-1], 
										MetadataCatalog.get("train"), 
										scale=1, 
										instance_mode=ColorMode.IMAGE_BW)
	v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
	img = v.get_image()[:,:,::-1]
	result.write(img)
cap.release()
result.release()
 

